In [15]:
import sys, os
import scipy.sparse as sps
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

sys.path.append(os.path.abspath('../kernel_inference'))
from new_york_inference import load

%matplotlib inline

In [2]:
kernel = load('../kernel_inference/filtered_paths.txt')

In [3]:
def get_matrix(kernel, dt):
    keys = set(kernel.keys())
    for ends in kernel.values():
        keys.update(ends.keys())
    keys = sorted(keys)

    mat = []
    for start in keys:
        row = []
        for end in keys:
            row.append(sum(1 for t in kernel[start][end] if t < dt))
        total = float(sum(row))
        mat.append([r / total for r in row] if total > 0 else [0] * len(keys))            
    return np.array(mat, np.float64)

slice = get_matrix(kernel, 360)
        

In [4]:
plt.hist(map(np.count_nonzero, slice))

(array([ 543.,  300.,  263.,  238.,  244.,  136.,   25.,   12.,    2.,    5.]),
 array([   0. ,   55.2,  110.4,  165.6,  220.8,  276. ,  331.2,  386.4,
         441.6,  496.8,  552. ]),
 <a list of 10 Patch objects>)

In [34]:
sc = plt.matshow(slice, cmap=cm.Blues,vmin=0.00001, vmax=1,norm=colors.LogNorm())
plt.colorbar(sc).set_label('Probability')
plt.show()

In [5]:
g = nx.from_numpy_matrix(slice)
lspec = nx.laplacian_spectrum(g)
aspec = nx.adjacency_spectrum(g)

/usr/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [35]:
plt.plot(sorted(aspec))
plt.show()
plt.plot(sorted(lspec))
plt.show()
print lspec[-2]

5.55276362425


/usr/lib/python2.7/site-packages/numpy/core/numeric.py:462: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [36]:
plt.plot(np.cumsum(sorted([a**2 for a in aspec])))
plt.show()

In [ ]:
plt.plot(sorted(np.linalg.svd(slice, compute_uv=0)))